# Demo 2 - Multi-Agent Team Interaction (Agent Collaboration)
By: [Lior Gazit](https://github.com/LiorGazit).  
Repo: [Agents-Over-The-Weekend](https://github.com/PacktPublishing/Agents-Over-The-Weekend/tree/main/Lior_Gazit/workshop_september_2025/)   
Running LLMs locally for free: This code leverages [`LLMPop`](https://pypi.org/project/llmpop/) that is dedicated to spinning up local or remote LLMs in a unified and modular syntax.  

<a target="_blank" href="https://colab.research.google.com/github/PacktPublishing/Agents-Over-The-Weekend/blob/main/Lior_Gazit/workshop_september_2025/codes_for_Lior_Bootcamp_talk_sept2025_demo2.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a> (pick a GPU Colab session for fastest computing)  

```
Disclaimer: The content and ideas presented in this notebook are solely those of the author, Lior Gazit, and do not represent the views or intellectual property of the author's employer.
```

Installing:

In [4]:
%pip -q install llmpop
%pip -q install sentence-transformers faiss-cpu langchain tiktoken langsmith langchain_openai -U "autogen-agentchat" "autogen-ext[openai]"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


**Imports:**

In [5]:
import os
import requests

In [6]:
from llmpop import init_llm
from langchain_core.prompts import ChatPromptTemplate

coder = init_llm(model="CodeLlama", provider="ollama", verbose=False)
reviewer = init_llm(model="llama3.2:1b", provider="ollama", verbose=False)

task = "Write a Python function to check if a number is prime."

# ---- Agent A: Coder ----
coder_prompt = ChatPromptTemplate.from_template(
    """
System: You are a coding agent. Output *code only*.
User task: {task}
"""
)

coder_result = (coder_prompt | coder).invoke({"task": task})
coder_code = getattr(coder_result, "content", str(coder_result)).strip()

# ---- Agent B: Reviewer ----
reviewer_prompt = ChatPromptTemplate.from_template(
    """
System: You are a strict code reviewer for Python.
Given the code below, evaluate for:
- Correctness and edge cases
- Time complexity reasonableness
- Readability (within reason, given constraints)
If changes are needed, provide a brief rationale followed by a *fully corrected* version.
If it's good as-is, say "APPROVED" and explain briefly why.

Code:
```python
{code}
```
"""
)

review_result = (reviewer_prompt | reviewer).invoke({"code": coder_code})
review_text = getattr(review_result, "content", str(review_result)).strip()

print("\n=== Coder output ===\n")
print(coder_code)

print("\n=== Reviewer feedback ===\n")
print(review_text)


RuntimeError: Ollama server at http://127.0.0.1:11434 failed to start within 15s.

Now, here is an example using AutoGen:

In [ ]:
import os
import asyncio
# In Colab, use getpass to securely prompt for your API key
from getpass import getpass
import openai

openai.api_key = getpass("Paste your OpenAI API key: ")

# 1. Import the agent classes and the OpenAI client
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient

async def multi_agent_demo():
    # 2. Configure your OpenAI API key
    api_key = openai.api_key
    if not api_key:
        openai.api_key = getpass("Paste your OpenAI API key: ")

    # 3. Create the OpenAI model client
    model_client = OpenAIChatCompletionClient(
        model="gpt-4o",
        api_key=api_key,
        temperature=0.0,
    )

    # 4. Instantiate two LLM agents with distinct roles
    coder = AssistantAgent(
        name="Coder",
        model_client=model_client,
        system_message="You are a Python coding assistant. Produce only working code."
    )
    reviewer = AssistantAgent(
        name="Reviewer",
        model_client=model_client,
        system_message="You are a code reviewer. Point out bugs or edge cases."
    )

    # 5. Coder agent writes a function
    code_task = "Write a Python function `is_prime(n)` that returns True if `n` is prime."
    code = await coder.run(task=code_task)
    print("=== Coder’s Output ===\n")
    for msg in code.messages:
        print(msg.content)

    # 6. Reviewer agent critiques the code
    review = await reviewer.run(task=f"Review the following code for correctness and edge cases:\n\n{code}")
    print("\n=== Reviewer’s Feedback ===\n")
    for msg in review.messages:
        print(msg.content)

    # 7. Clean up
    await model_client.close()

# 8. Execute the multi‑agent demo
await multi_agent_demo()